In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# FINAL PROJECT: Import DATABASE
city_data_df = pd.read_csv("final_covid_tourist.csv")
city_data_df.head()

,city,provine_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,projected_covid_cases
0,NaN,Andalusia,Spain,37.544300,-4.727800,724716,703791.0,2021-08-05,28255949,"[37.5443,-4.7278]",NaN
1,NaN,Aragon,Spain,41.597600,-0.905700,149510,142137.0,2021-08-05,28255949,"[41.5976,-0.9057]",NaN
2,NaN,Asturias,Spain,43.361400,-5.859300,67369,64296.0,2021-08-05,28255949,"[43.3614,-5.8593]",NaN
3,NaN,Baleares,Spain,39.710358,2.995148,85692,83289.0,2021-08-05,28255949,"[39.710358,2.995148]",NaN
4,NaN,C. Valenciana,Spain,39.484000,-0.753300,472605,455155.0,2021-08-05,28255949,"[39.484,-0.7533]",NaN


In [4]:
city_data_df.dtypes

city                        object
provine_state               object
country_region              object
lat                        float64
long                       float64
confirmed_cases              int64
cases_per_day              float64
last_update                 object
people_fully_vaccinated      int64
coordinates                 object
projected_covid_cases      float64
dtype: object

In [8]:
# ??? ARE WE KEEPING THIS OR PROMPT FOR LOCATION INSTEAD?
# 2. Prompt the user to enter location
location = str(input("Where do you want to go for your trip? "))

In [10]:
# FINAL PROJECT: FILTER THE DF USING LOC METHOD
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["country_region"] == location)]
preferred_cities_df.head(10)

,city,provine_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,projected_covid_cases
74,NaN,NaN,Hungary,47.1625,19.5033,809731,29926.0,2021-08-05,5480218,"[47.1625,19.5033]",NaN
3458,NaN,NaN,Hungary,47.1625,19.5033,809731,29926.0,2021-08-05,5480218,"[47.1625,19.5033]",NaN
6841,NaN,NaN,Hungary,47.1625,19.5033,809731,29926.0,2021-08-05,5480218,"[47.1625,19.5033]",NaN


In [19]:
# 4a. Determine if there are any empty rows.
# preferred_cities_df.isnull().sum()

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# clean_df = preferred_cities_df.dropna()
# clean_df

In [12]:
# FINAL PROJECT: USE BELOW BUT ADD CURRENT AND PROJECTED COVID CASES
# hotel_df = clean_df[["City", "Country", "Current Covid Cases", "Projected Covid Cases", "Current Description", "Lat", "Lng"]].copy()
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# hotel_df["Hotel Name"] = preferred_cities_df
hotel_df = preferred_cities_df[["country_region", "city", "provine_state", "lat", "long", "confirmed_cases", "people_fully_vaccinated", "projected_covid_cases"]].copy()
hotel_df

,country_region,city,provine_state,lat,long,confirmed_cases,people_fully_vaccinated,projected_covid_cases
74,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN
3458,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN
6841,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN


In [13]:
# Add Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df

,country_region,city,provine_state,lat,long,confirmed_cases,people_fully_vaccinated,projected_covid_cases,Hotel Name
74,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,
3458,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,
6841,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["lat"]
    lng = row["long"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,country_region,city,provine_state,lat,long,confirmed_cases,people_fully_vaccinated,projected_covid_cases,Hotel Name


In [17]:
# FINAL PROJECT: CREATE OUTPUT FILE
# output_data_file = "ENTER NEW FILE.csv"
# 8a. Create the output File (CSV)
output_data_file = "covid_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")
hotel_df.head()

,country_region,city,provine_state,lat,long,confirmed_cases,people_fully_vaccinated,projected_covid_cases,Hotel Name
74,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,Порчальма
3458,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,Порчальма
6841,Hungary,NaN,NaN,47.1625,19.5033,809731,5480218,NaN,Порчальма


In [18]:
# FINAL PROJECT: UPDATE TEMPLATE TO INCLUDE COVID CASES AND PROJECTED COVID CASES
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City Name</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Current Weather</dt><dd>{Current Description}</dd>
# <dt>Current Covid Cases</dt><dd>{CURRENT COVID CASES}</dd>
# <dt>Projected Covid Cases</dt><dd>{PROJECTED COVID CASES}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_region}</dd>
<dt>Confirmed Covid Cases</dt><dd>{confirmed_cases}</dd>
<dt>Projected Covid Cases</dt><dd>{projected_covid_cases}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["lat", "long"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["lat", "long"]]
covid = hotel_df["confirmed_cases"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=covid, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))